## Car Price Predictor using Random Forest Regression

# Used Car Price Prediction

## 1) Problem statement.

* This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
* If user can predict the price of the car based on input features.
* Prediction results can be used to give new seller the price suggestion based on market condition.

## 2) Data Collection.
* The Dataset is collected from scrapping from cardheko webiste
* The data consists of 13 column and 15411 rows.

In [1]:
# Importing necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load Dataset
df=pd.read_csv('cardekho_imputated.csv')
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Feature Engineering

### Data Cleaning
* Handling Missing values 
* Handling Duplicates
* Check data type
* Understand the dataset

In [5]:
# Dropping Unnecessary columns
df.drop(['Unnamed: 0','car_name','brand'],axis=1,inplace=True)

In [6]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [13]:
# Detecting Different Features
# Categorical Features
cat_features=[feature for feature in df.columns if df[feature].dtype=='O']
# Numerical Features
num_features=[feature for feature in df.columns if df[feature].dtype!='O']
# Discrete Features
discrete_features=[feature for feature in num_features if len(df[feature].unique())<=25]
# Continuous Features
continuous_features=[feature for feature in num_features if feature not in discrete_features]

print("Categorical Features: ",cat_features)
print("Numerical Features: ",num_features)
print("Discrete Features: ",discrete_features)
print("Continuous Features: ",continuous_features)


Categorical Features:  ['model', 'seller_type', 'fuel_type', 'transmission_type']
Numerical Features:  ['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
Discrete Features:  ['vehicle_age', 'seats']
Continuous Features:  ['km_driven', 'mileage', 'engine', 'max_power', 'selling_price']


In [14]:
# Missing Values
df.isnull().sum()

model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [15]:
# Independent and Dependent Features
X=df.drop('selling_price',axis=1)
Y=df['selling_price']

In [16]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [17]:
Y

0         120000
1         550000
2         215000
3         226000
4         570000
          ...   
15406     250000
15407     925000
15408     425000
15409    1225000
15410    1200000
Name: selling_price, Length: 15411, dtype: int64

## Feature Encoding and Scaling
**One Hot Encoding for Columns which had lesser unique values and not ordinal**
* One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.

In [18]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [19]:
df['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [20]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [21]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [24]:
df['seller_type'].value_counts()

seller_type
Dealer              9539
Individual          5699
Trustmark Dealer     173
Name: count, dtype: int64

In [25]:
df['fuel_type'].value_counts()

fuel_type
Petrol      7643
Diesel      7419
CNG          301
LPG           44
Electric       4
Name: count, dtype: int64

In [26]:
df['transmission_type'].value_counts()

transmission_type
Manual       12225
Automatic     3186
Name: count, dtype: int64

In [40]:
# Create Column Transformer
num_features=X.select_dtypes(exclude="object").columns
ohe_fetaures=['seller_type','transmission_type','fuel_type']

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
ohe=OneHotEncoder(drop='first')
standardscaler=StandardScaler()

preprocessing=ColumnTransformer(
    [('One Hot Encoding',ohe,ohe_fetaures),
     ('Standard Scaling',standardscaler,num_features)],
     remainder="passthrough"
)


In [41]:
preprocessing

ColumnTransformer(remainder='passthrough',
                  transformers=[('One Hot Encoding',
                                 OneHotEncoder(drop='first'),
                                 ['seller_type', 'transmission_type',
                                  'fuel_type']),
                                ('Standard Scaling', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])

In [42]:
X=preprocessing.fit_transform(X)

In [33]:
X.shape

(15411, 10)

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
X_train.shape,Y_train.shape

((12328, 14), (12328,))

## Model Training and Model Selection

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [45]:
# A function to evaluate model
def evaluate(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2score=r2_score(true,predicted)
    return mae,rmse,r2score


In [46]:
models={
    'Linear':LinearRegression(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'K-Neighbors Regressor':KNeighborsRegressor(),
    'Decision Tree':DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor()
}

In [50]:
for i in range (len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

    # Training Data
    Y_train_pred=model.predict(X_train)
    mae_train,rmse_train,r2score_train=evaluate(Y_train,Y_train_pred)

    # test Data
    Y_test_pred=model.predict(X_test)
    mae_test,rmse_test,r2score_test=evaluate(Y_test,Y_test_pred)

    print(f"------Performance for Model {list(models.keys())[i]}--------")
    print("Performance for Training Data")
    print("Mean Absolute Error: ",mae_train)
    print("Root Mean Squared Error: ",rmse_train)
    print("R Square: ",r2score_train)

    print("Performance for Test Data")
    print("Mean Absolute Error: ",mae_test)
    print("Root Mean Squared Error: ",rmse_test)
    print("R Square: ",r2score_test)


------Performance for Model Linear--------
Performance for Training Data
Mean Absolute Error:  268101.60708299326
Root Mean Squared Error:  553855.6665411663
R Square:  0.6217719576765959
Performance for Test Data
Mean Absolute Error:  279618.5794158424
Root Mean Squared Error:  502543.5930230984
R Square:  0.6645109298852006
------Performance for Model Ridge--------
Performance for Training Data
Mean Absolute Error:  268059.801468831
Root Mean Squared Error:  553856.3159709624
R Square:  0.6217710706848424
Performance for Test Data
Mean Absolute Error:  279557.2168930274
Root Mean Squared Error:  502533.8229890288
R Square:  0.6645239743566811
------Performance for Model Lasso--------
Performance for Training Data
Mean Absolute Error:  268099.22267038835
Root Mean Squared Error:  553855.6709544364
R Square:  0.6217719516489515
Performance for Test Data
Mean Absolute Error:  279614.7460664388
Root Mean Squared Error:  502542.6696781151
R Square:  0.6645121627010977
------Performance fo

In [51]:
#Initialize few parameter for Hyperparamter tuning
knn_params = {"n_neighbors": [2, 3, 10, 20, 40, 50]}
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

In [52]:
randomcv_models=[('KNN',KNeighborsRegressor(),knn_params),
                 ('RF',RandomForestRegressor(),rf_params)]

In [54]:
from sklearn.model_selection import RandomizedSearchCV
for name,model_est,params in randomcv_models:
    random=RandomizedSearchCV(estimator=model_est,param_distributions=params,
                             n_jobs=-1,verbose=2,n_iter=100,cv=3)
    random.fit(X_train,Y_train)


    print(f"------Best Params for Model {name}--------")
    
    print(random.best_params_)


Fitting 3 folds for each of 6 candidates, totalling 18 fits


------Best Params for Model KNN--------
{'n_neighbors': 10}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
------Best Params for Model RF--------
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 5, 'max_depth': 15}


In [55]:
# Retaining the models with best params
models={
    'K-Neighbors Regressor':KNeighborsRegressor(n_neighbors=10,n_jobs=-1),
    'Random Forest Regressor':RandomForestRegressor(n_estimators= 100, min_samples_split= 2, max_features= 5, max_depth= 15,n_jobs=-1)
}
for i in range (len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

    # Training Data
    Y_train_pred=model.predict(X_train)
    mae_train,rmse_train,r2score_train=evaluate(Y_train,Y_train_pred)

    # test Data
    Y_test_pred=model.predict(X_test)
    mae_test,rmse_test,r2score_test=evaluate(Y_test,Y_test_pred)

    print(f"------Performance for Model {list(models.keys())[i]}--------")
    print("Performance for Training Data")
    print("Mean Absolute Error: ",mae_train)
    print("Root Mean Squared Error: ",rmse_train)
    print("R Square: ",r2score_train)

    print("Performance for Test Data")
    print("Mean Absolute Error: ",mae_test)
    print("Root Mean Squared Error: ",rmse_test)
    print("R Square: ",r2score_test)

------Performance for Model K-Neighbors Regressor--------
Performance for Training Data
Mean Absolute Error:  103471.61745619727
Root Mean Squared Error:  363460.7006081945
R Square:  0.8371171210295787
Performance for Test Data
Mean Absolute Error:  117487.13104119364
Root Mean Squared Error:  263882.25816523493
R Square:  0.9074979668016598
------Performance for Model Random Forest Regressor--------
Performance for Training Data
Mean Absolute Error:  57059.499091562
Root Mean Squared Error:  139547.64325032762
R Square:  0.9759892607897944
Performance for Test Data
Mean Absolute Error:  98575.41006953461
Root Mean Squared Error:  215092.23805111684
R Square:  0.9385417242357296
